In [1]:
import requests
import pandas as pd

In [2]:
nome_area = 'RECANTO VERDE II'
sigla_area = 'SM-05'

In [3]:
colours = pd.Series({'R1':'#4FC26A','R2':'#F0E113','R3':'#FF8801','R4':'#BF243C'})
colours.name = 'colours'
colours.index.name = 'grau_risco'

In [4]:
url_levantamento = f'https://uzu2spnwitelgca-db202004101957.adb.sa-saopaulo-1.oraclecloudapps.com/ords/areas_risco/Relatorios_dem/levantamento?nome_var={nome_area}&sigla_var={sigla_area}'
r_levantamento = requests.get(url_levantamento)
json_levantamento = r_levantamento.json()
lev = pd.DataFrame(json_levantamento['items'])

In [5]:
lev

,nome,sigla,setor,grau_risco,moradias,moradores,familias,criancas,idosos,pcds
0,RECANTO VERDE II,SM-05,SM-05-01 (R3),R3,264,756,266,60,22,8
1,RECANTO VERDE II,SM-05,SM-05-02 (R3),R3,186,560,185,37,17,2
2,RECANTO VERDE II,SM-05,SM-05-03 (R2),R2,845,2482,846,181,74,17
3,RECANTO VERDE II,SM-05,SM-05-04 (R2),R2,667,1959,668,143,59,14
4,RECANTO VERDE II,SM-05,SM-05-05 (R1),R1,1109,3257,1111,238,98,23


In [6]:
# Get data for Moradias x Setor dados_setor_moradias
dados_setor_moradias = lev[['setor', 'moradias']]
dados_setor_moradias.columns = ['Setor','Quantidade de Moradias']
dados_setor_moradias

,Setor,Quantidade de Moradias
0,SM-05-01 (R3),264
1,SM-05-02 (R3),186
2,SM-05-03 (R2),845
3,SM-05-04 (R2),667
4,SM-05-05 (R1),1109


In [7]:
# Get data for Moradias x Risco dados_risco_moradias
dados_risco_moradias = lev[['grau_risco', 'moradias']].groupby('grau_risco').sum().reset_index()
dados_risco_moradias.columns = ['Risco','Quantidade de Moradias']
dados_risco_moradias = pd.merge(dados_risco_moradias, colours, left_on='Risco', right_index=True)
dados_risco_moradias
    

,Risco,Quantidade de Moradias,colours
0,R1,1109,#4FC26A
1,R2,1512,#F0E113
2,R3,450,#FF8801


In [8]:
# Get data for Moradores x Setor dados_setor_moradores
dados_setor_moradores = lev[['setor', 'moradores']]
dados_setor_moradores.columns = ['Setor','Quantidade de Moradores']
dados_setor_moradores

,Setor,Quantidade de Moradores
0,SM-05-01 (R3),756
1,SM-05-02 (R3),560
2,SM-05-03 (R2),2482
3,SM-05-04 (R2),1959
4,SM-05-05 (R1),3257


In [9]:
colours.keys()

Index(['R1', 'R2', 'R3', 'R4'], dtype='object', name='grau_risco')

In [10]:
# Get data for Moradores x Risco
lev[['grau_risco', 'moradores']].groupby('grau_risco').sum()

,moradores
grau_risco,
R1,3257
R2,4441
R3,1316


In [11]:
colours.reset_index()

,grau_risco,colours
0,R1,#4FC26A
1,R2,#F0E113
2,R3,#FF8801
3,R4,#BF243C


In [12]:
lev[['grau_risco', 'moradores']].groupby('grau_risco').sum().reset_index()

,grau_risco,moradores
0,R1,3257
1,R2,4441
2,R3,1316


In [13]:
risco_df_moradores = pd.merge(colours.reset_index()['grau_risco'], lev[['grau_risco', 'moradores']].groupby('grau_risco').sum().reset_index(), how='left')
risco_df_moradores.fillna(0, inplace=True)
risco_df_moradores['moradores'] = risco_df_moradores['moradores'].astype(int)
risco_df_moradores.set_index('grau_risco').to_dict()['moradores']

{'R1': 3257, 'R2': 4441, 'R3': 1316, 'R4': 0}

In [14]:
dados = {}

In [15]:
#dados['total_familias'] = get_familias(dados_fichas_uma_area)

dados['total_familias'] = lev['familias'].sum()

In [16]:
#     dados['total_criancas'] = dados_fichas_uma_area['CRIANCA_CSA'].sum()
dados['total_criancas'] = lev['criancas'].sum()

In [17]:
dados['total_idosos'] = lev['idosos'].sum()
dados['total_pcds'] = lev['pcds'].sum()

In [26]:
risco_df_moradias = dados_risco_moradias.drop(columns='colours')
risco_df_moradias.columns = ['grau_risco', 'moradias']

In [41]:
dados['risco_df'] = risco_df_moradores.merge(risco_df_moradias, how='left').set_index('grau_risco')
dados['risco_df'].columns = ['Moradias', 'Habitantes']
dados['risco_df'].fillna(0, inplace=True)
dados['risco_df']['Habitantes'] = dados['risco_df']['Habitantes'].astype(int)
dados['risco_df']

,Moradias,Habitantes
grau_risco,,
R1,3257,1109
R2,4441,1512
R3,1316,450
R4,0,0
